In [4]:
import cv2
import sys,os
import numpy as np
# import rosbag
# import rospy
# from sensor_msgs.msg import Image
# from cv_bridge import CvBridge
# from pyquaternion import Quaternion
from numpy.linalg import inv, norm
from scipy.spatial.transform import Rotation as R, Slerp
from shutil import copyfile
import sqlite3



In [6]:
#1. transfrom vi map pose to camera frame and save as colmap format
#2. copy corresponding images to colmap directory
baseDir = "/home/pang/colmap_ws/office/office-1-5"
tmp = np.loadtxt(baseDir+"/vertices.csv", dtype=np.str, delimiter=",")
tmp = tmp[1:]
viMap=[]
imgDir = baseDir+"/fisheye1/"
imgTime = np.loadtxt(baseDir+"/fisheye_timestamps.txt",usecols=(2,))*1000#ms outputDir

sfmImagesTxt = open(baseDir + "/images.txt",'w')
outputDir = baseDir +"/sfm1"
imgList = os.listdir(imgDir)
imgList.sort()
if not os.path.isdir(outputDir):
    os.makedirs(outputDir)
else:
    filelist = os.listdir(outputDir)
    for img in filelist:
        os.remove("{}/{}".format(outputDir,img))   
        
selectedIdx=[]
selectedPose=[]
for frame in tmp:
    viMap.append([float(a) for a in frame])
for v in viMap:
    idx = np.argmin(np.abs(v[1] - imgTime))
    if np.abs(imgTime[idx] - v[1]) < 50000000:
        selectedIdx.append(idx)
        selectedPose.append(v)

for i in range(len(selectedPose)):
    p = selectedPose[i]
    T_W_C = np.eye(4)
    T_W_C[:3,3] = np.array([p[2],p[3],p[4]])
    T_W_C[:3,:3] = (R.from_quat([p[5],p[6],p[7],p[8]])).as_dcm()
    
    
    T_C_W = np.linalg.inv(T_W_C)
    quat = (R.from_dcm(T_C_W[:3,:3])).as_quat()
    t = T_C_W[:3,3]
    sfmImagesTxt.write("{} {:.6f} {:.6f} {:.6f} {:.6f} {:.6f} {:.6f} {:.6f} {} {:0>5}.png \n\n".
                       format(i+1, quat[3],quat[0],quat[1],quat[2],t[0], t[1],t[2],1,i+1))
    copyfile(imgDir+"/"+str(imgList[selectedIdx[i]]), outputDir+"/"+ "{:0>5}.png".format(i+1))
sfmImagesTxt.close()

